#### 1. DB 연결하기

In [1]:
from dotenv import load_dotenv
import os
import psycopg2
import io
import pandas as pd
import time

In [2]:
load_dotenv('.env')
db_name = os.getenv('db_name')
user_name = os.getenv('db_admin')
password = os.getenv('password')

conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database=db_name,
    user=user_name,
    password=password
)

cur = conn.cursor()
cur.execute("SELECT current_database();")
print(cur.fetchone())

('saturnbird',)


In [3]:
import os
import time

full_path = r'C:\practice_db\csv_file\\'
row_count = 0

csv_files = [f for f in os.listdir(full_path) if f.endswith('.csv')]

start_time = time.perf_counter()

for file_name in csv_files:
    try:
        table_name = file_name.replace('데이터_','').replace('테이블.csv','')

        file_path = os.path.join(full_path, file_name)

        # 행 수 미리 계산 (헤더 제외)
        with open(file_path, 'r', encoding='utf-8') as f:
            file_rows = sum(1 for _ in f) - 1

        # COPY 실행
        with open(file_path, 'r', encoding='utf-8') as f:
            cur.copy_expert(
                f"COPY public.{table_name} FROM STDIN WITH CSV HEADER",
                f
            )

        row_count += file_rows
        print(f"{table_name} → {file_rows} rows inserted")

    except Exception as e:
        print(f"{file_name} 실패:", e)

# 루프 끝난 후 한 번만 commit (더 빠름)
conn.commit()

end_time = time.perf_counter()

print("총 입력 행:", row_count)
print("총 소요 시간:", round(end_time - start_time, 2), "초")
print("초당 처리 행 수:", int(row_count / (end_time - start_time)))

개인문의 → 150000 rows inserted
결제 → 2999979 rows inserted
관리자 → 50 rows inserted
리뷰 → 500000 rows inserted
배송 → 2999979 rows inserted
상품문의 → 150000 rows inserted
상품 → 50000 rows inserted
이벤트 → 19 rows inserted
장바구니 → 100000 rows inserted
재고 → 50000 rows inserted
주문상세 → 6999949 rows inserted
주문 → 2999979 rows inserted
최근본상품 → 100000 rows inserted
쿠폰이력 → 983288 rows inserted
쿠폰 → 300 rows inserted
포인트 → 350000 rows inserted
회원 → 99951 rows inserted
총 입력 행: 18533494
총 소요 시간: 149.71 초
초당 처리 행 수: 123794
